# Introduction à la programmation Python

Julien Velcin, Université Lyon 2

Formation doctorale de l'UdL (2023-2024)

Ce notebook est un exemple de traitement des données textuelles à des fins de classification binaires (2 classes uniquement).

In [ ]:
# libraires utilisées

import numpy as np
import pandas as pd

# prétraitement et évaluation
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# différents classifieurs
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier

In [ ]:
# pour ignorer les avertissements

import warnings
warnings.filterwarnings('ignore')

# chargement des données

Chargement des données à partir d'un fichier .csv (colonnes séparées par une tabulation = "\t").

Nous allons utiliser, pour la démonstration, un jeu de données contenant des propos toxiques diffusés sur Internet (cf. défi Kaggle de 2018 : https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge).

**Attention :** certains propos tenus dans les données peuvent choquer (insultes, racisme, etc.). Si vous le préferez, rien ne vous oblige à aller lire les textes d'origine.

Chargez les données dans une variable *data_load* et afficher les dimensions de la table.

In [ ]:
# à compléter

Affichez un extrait des données (fin de la table).

In [ ]:
# à compléter

Affectez la liste des textes dans une variable *textes* et la liste des classes dans une variable *classes*.

In [ ]:
# à compléter

Affichez la distribution des valeurs pour les deux classes. Pour cela, vous pouvez utiliser la méthode *unique* de la librairie **numpy**.

In [ ]:
# à compléter

# Mise en forme des données

Afin de pouvoir classer automatiquement ces données textuelles, nous allons passer par l'utilisation de la librairie **scikit-learn** et en particulier par l'outil nommé *TfidfVectorizer* :

In [ ]:
# les paramètres sont un peu fixés au hasard ici
vectorize = TfidfVectorizer(ngram_range=(1, 2), max_df=0.5, min_df=5)
vectorize.fit(textes)
data = vectorize.transform(textes)

On divise le jeu de données initial en deux sous-ensembles :

- données d'entraînement ou *training set*
- données de test (ici appelé parfois valiation)

In [ ]:
subtrain_X, subtest_X, subtrain_y, subtest_y = train_test_split(data, classes,
                                                    train_size=0.7,
                                                    test_size=0.3,
                                                    random_state=42,
                                                   stratify=classes)

In [ ]:
# vérifions que la stratification a bien fonctionné

unique_values_a, counts_a = np.unique(subtrain_y, return_counts=True)
unique_values_b, counts_b = np.unique(subtest_y, return_counts=True)

print(f"Jeu d'entrainement : " + " ".join([f"{value}:{count}" for value, count in zip(unique_values_a, counts_a)]))
print(f"Jeu de test : " + " ".join([f"{value}:{count}" for value, count in zip(unique_values_b, counts_b)]))   

In [ ]:
subtrain_X.shape

# Apprentissage automatique

Nous allons tester différents algorithmes de classification supervisée qui traitent ici les textes en suivant l'hypothèse du sac de mots (BoW).

## Simple régression logistique

In [ ]:
lr = LogisticRegression()

lr.fit(subtrain_X, subtrain_y)
pred_y_lr = lr.predict(subtest_X)

In [ ]:
print("Ce qui est prédit : " + str(pred_y_lr[0:20]))

print("La vérité : " + str(subtest_y[0:20]))

In [ ]:
res_test = np.sum(pred_y_lr == subtest_y) / float(len(subtest_y))
                                       
print(f"Réussite en validation {res_test:.1%}")

A comparer à la réussite sur le jeu d'entraînement :

In [ ]:
pred_y_lr = lr.predict(subtrain_X)

res_train = np.sum(pred_y_lr == subtrain_y) / float(len(subtrain_y))
print(f"Réussite sur les données d'entraînement {res_train:.1%}")

Deux remarques importantes à ce stade :

    1. On peut observer une différence entre les deux erreurs, signe d'un sur-apprentissage (*overfitting*)
    2. Nous avons utilisé un même nom de variable (pred_y_lr) pour deux résultats différents : c'est une mauvaise habitude et nous allons utiliser 2 variables différentes pour la suite.

Concernant le sur-apprentissage, une piste serait ici de régulariser le modèle avec un terme qui pénalise une trop grande dispersion des poids (régression ridge et lasso).
Essayons par ex. la régression ridge :

In [ ]:
lr_ridge = RidgeClassifier(alpha=20.0)
lr_ridge.fit(subtrain_X, subtrain_y)
pred_train_lr_ridge = lr_ridge.predict(subtrain_X)
pred_test_lr_ridge = lr_ridge.predict(subtest_X)

Le paramètre *alpha* indique la force de la régularisation.

In [ ]:
res_train = np.sum(pred_train_lr_ridge == subtrain_y) / float(len(subtrain_y))
res_test = np.sum(pred_test_lr_ridge == subtest_y) / float(len(subtest_y))

print(f"Réussite (accuracy) sur :")
print(f"  - ensemble d'entraînement : {res_train:.1%}")
print(f"  - ensemble de test : {res_test:.1%}")

On constate qu'on diminue le sur-apprentissage, mais au prix d'une réussite plus faible...

## Une évaluation plus robuste : la validation croisée

Il s'agit ici d'évaluation la capacité de l'algorithme choisi à résoudre la tâche de classification.
L'objectif est donc plus de choisir l'algorithme que de trouver le modèle lui-même.

In [ ]:
seed = 7
np.random.seed(seed)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

results = cross_val_score(lr, subtrain_X, subtrain_y, cv=kfold)
print(f"Validation croisée : moyenne {results.mean():.1%} et écart-type {results.std():.2f}")

Une fois l'algorithme choisi, il s'agit de réentraîner le modèle sur l'ensemble des données d'entraînement (cf. 3.1).

Pour simplifier le code, on crée une fonction d'affichage des deux erreurs :

In [ ]:
def print_accuracy(nom_algo, pred_train, pred_test):
    print(nom_algo + " : ")
    acc_app = np.sum(pred_train == subtrain_y) / float(len(subtrain_y))
    print(f"  - réussite (accuracy) apparente : {acc_app:.1%}")
    acc_gen = np.sum(pred_test == subtest_y) / float(len(subtest_y))
    print(f"  - réussite (accuracy) en généralisation : {acc_gen:.1%}")                                                          

A présent, il est possible de suivre la même méthodologie mais en changeant l'algorithme de classification. La librairie propose ainsi les classifieurs suivants :

- Machines à vecteurs supports (ou SVM)
- Arbres de décisions (en particulier XGBoost)
- Réseaux de neurones artificiels
- KNN
- Naives Bayes
...

## Ce que vous pouvez faire

* Essayez d'utiliser plusieurs de ces classifieurs en suivant le même schéma que ce que nous avons vu avec la régression logistique.
* Une fois que vous avez des résultats obtenus par plusieurs algorithmes, résumez ces résultats sous la forme d'une table pandas.
* Le schéma étant toujours le même (fit et predict, mesures d'évaluation), factorisez le code pour le rendre plus modulaire, si possible en externalisant le code dans un fichier .py.